# Sandbox

In [67]:
import sys
sys.path.append('../sddip')

In [68]:
#%run ../scripts/create_result_directories.py

In [69]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats


import sddip.tree as tree
import sddip.storage as storage
import sddip.utils as utils
import sddip.config as config
import sddip.dualsolver as dualsolver
import sddip.ucmodel as ucmodel


## Data Processing

In [70]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")

scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

### Bus Data

In [71]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


In [72]:
bus_df.dtypes

bus_i       int64
type        int64
Pd          int64
Qd          int64
Gs          int64
Bs          int64
area        int64
Vm        float64
Va          int64
baseKV      int64
zone        int64
Vmax      float64
Vmin      float64
dtype: object

### Branch Data

In [73]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


### Generator Data

In [74]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [75]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [76]:
scenario_df
# TODO Stochastische Nachfrage für jeden Knoten

,t,n,p,Pd
0,1,1,1.0,350
1,2,1,0.5,300
2,2,2,0.5,200


### Power Transfer Distribution Factor

In [77]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

### Parameter Organization

In [78]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

probs = np.empty(n_stages, dtype=object)
# p_d[t,n]
p_d = np.empty(n_stages, dtype=object)

for t in range(n_stages):
    stage_df = scenario_df.loc[scenario_df["t"] == t+1]
    probs[t] = np.array(stage_df.p)
    p_d[t] = np.array(stage_df.Pd)

########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = np.array([probs[t].dot(p_d[t]) for t in range(n_stages)])

## Recombining Tree

In [79]:
# n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()

# node_params_df = scenario_df.drop(["t", "n"], axis=1)
# node_params_dicts = node_params_df.to_dict("records")

# s_tree = tree.RecombiningTree(n_nodes_per_stage)

# s_tree.params["gc"] = gen_cost_df.c1
# s_tree.params["suc"] = gen_cost_df.startup
# s_tree.params["sdc"] = gen_cost_df.startup
# s_tree.params["pg_min"] = gen_df.Pmin
# s_tree.params["pg_max"] = gen_df.Pmax
# s_tree.params["pl_max"] = branch_df.rateA
# s_tree.params = {k: v.values.tolist() for k,v in s_tree.params.items()}
# s_tree.params["ptdf"] = ptdf
# s_tree.params["n_gens"] = len(s_tree.params["gc"])
# s_tree.params["n_buses"] = ptdf.shape[1]

# for stage in s_tree:
#     for node in stage:
#         params = node_params_dicts.pop(0)
#         node.prob = params.pop("p")
#         node.params = params


# for stage in s_tree:
#     stage.params["ex_Pd"] = np.zeros((1,s_tree.params["n_buses"]))
#     for node in stage:
#         stage.params["ex_Pd"] += node.prob*np.array(node.params["Pd"])

## SDDiP

### Forward pass

#### Model

In [80]:
penalty = 10000
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist() + [penalty]*2
present_demand = p_d[0][0]
theta_lb = 0
trial_point = [0]*n_gens


uc_fw = ucmodel.ModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

uc_fw.add_objective(cost_coeffs)

uc_fw.add_balance_constraints(present_demand)

uc_fw.add_generator_constraints(pg_min, pg_max)

uc_fw.add_power_flow_constraints(ptdf, pl_max)

uc_fw.add_startup_shutdown_constraints()

# TODO Ramp rate constraints

uc_fw.add_copy_constraints(trial_point)

uc_fw.add_cut_constraints([theta_lb],[0])

#uc_fw.model.setParam("OutputFlag", 0)

#uc_fw.model.display()

#### Run

In [81]:
# TODO Sampling
samples = [[0,1], [0,0]]
n_samples = len(samples)

v_opt_k = []
i = 0

solution_storage = storage.SolutionStorage()

for k in range(n_samples):
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        # Update iteration-, state-, and realization-specific constraints
        uc_fw.update_balance_constraints(p_d[t][n])

        uc_fw.update_copy_constraints(trial_point)

        # TODO Update Cut constraints
        #uc_fw.update_cut_constraints()
        
        # Solve problem
        uc_fw.model.optimize()
        uc_fw.model.printAttr("X")

        # Store xtik, ytik, ztik, vtik
        y_kt = [y_g.x for y_g in uc_fw.y]
        z_kt = [z_g.x for z_g in uc_fw.z]
        s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
        s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]
        
        # Value of stage t objective function
        v_opt_kt = uc_fw.model.getObjective().getValue() - uc_fw.theta.x

        v_opt_k[-1] += v_opt_kt

        # New trial point
        x_trial = [x_g.x for x_g in uc_fw.x]

        solution = {
            "x": x_trial,
            "y": y_kt,
            "z": z_kt
        }

        solution_storage.add_solution(i, k, t, solution)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9 rows, 8 columns and 15 nonzeros
Model fingerprint: 0xfbd9ed6e
Variable types: 5 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 1e+06]
Presolve removed 8 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 7.000000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     700.0000000  700.00000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 ava

In [82]:
########################################################################################################################
# Forward pass
########################################################################################################################

x = np.empty(n_gens, dtype=object)
y = np.empty(n_gens, dtype=object)
z = np.empty(n_gens, dtype=object)
s_up = np.empty(n_gens, dtype=object)
s_down = np.empty(n_gens, dtype=object)
theta = None
ys_p = None
ys_n = None

cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist()

penalty = 10000

# Cut variable upper bounds
theta_lb = np.full(n_stages, 10000)

# Cut intercepts and cut gradients
ci = np.array([theta_lb])
cg = np.array([[]])

x_trial = np.zeros(n_gens)

samples = [[0,1], [0,0]]
n_samples = len(samples)
v_opt_k =[]

# Solution storage
solution_storage = storage.SolutionStorage()
i = 0
for k in range(n_samples):
    x_trial = np.zeros(n_gens)
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        model = gp.Model("P{}".format(t))

        #Variables
        for g in range(n_gens):
            x[g] = model.addVar(vtype = gp.GRB.BINARY, name = "x_%i,%i"%(g+1,t+1))
            y[g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "y_%i,%i"%(g+1,t+1))
            z[g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, ub = 1, name = "z_%i,%i"%(g+1,t+1))
            s_up[g] = model.addVar(vtype = gp.GRB.BINARY, name = "s_up_%i,%i"%(g+1,t+1))
            s_down[g] = model.addVar(vtype = gp.GRB.BINARY, name = "s_down_%i,%i"%(g+1,t+1))
        
        theta = model.addVar(vtype = gp.GRB.CONTINUOUS, name = "cv_%i"%(t+1))
        ys_p = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "ys_p_%i"%(t+1))
        ys_n = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "ys_n_%i"%(t+1))

        model.update()

        # Objective
        obj_vars = y.tolist() + s_up.tolist() + s_down.tolist()
        obj = gp.LinExpr(cost_coeffs, obj_vars)
        obj += penalty*(ys_p+ ys_n)
        obj += theta
        model.setObjective(obj)

        # Balance constraints
        model.addConstr((gp.quicksum(y) + ys_p - ys_n == p_d[t][n]), "balance")

        # Generator constraints
        model.addConstrs((y[g] >= pg_min[g]*x[g] for g in range(n_gens)), "gen_min")
        model.addConstrs((y[g] <= pg_max[g]*x[g] for g in range(n_gens)), "gen_max")

        # Power flow constraints
        model.addConstrs((gp.quicksum(ptdf[l,b] * gp.quicksum(y[g] for g in gens_at_bus[b]) for b in range(n_buses)) 
            <= pl_max for l in range(n_lines)), "flow(1)")
        model.addConstrs((-gp.quicksum(ptdf[l,b] * gp.quicksum(y[g] for g in gens_at_bus[b]) for b in range(n_buses)) 
            <= pl_max for l in range(n_lines)), "flow(2)")

        # Startup and shutdown constraints    
        model.addConstrs((x[g] - z[g] <= s_up[g]  for g in range(n_gens)), "up-down(1)")
        model.addConstrs((x[g] - z[g] <= s_up[g] - s_down[g]  for g in range(n_gens)), "up-down(2)")

        # TODO Ramp rate constraints

        # Copy constraints
        model.addConstrs((z[g] == x_trial[g] for g in range(n_gens)), "copy")

        # Cut constraints
        # TODO Calculate cut gradient and intercept
        model.addConstrs((theta >= intcp + grad.T.dot(x) for intcp, grad in zip(ci[0], cg[0])), "cut")


        model.update()

        model.optimize()

        # Store xtik, ytik, ztik, vtik
        y_kt = [y[g].x for g in range(n_gens)]
        s_up_kt = [s_up[g].x for g in range(n_gens)]
        s_down_kt = [s_down[g].x for g in range(n_gens)]
        z_kt = [z[g].x for g in range(n_gens)]

        obj_var_opt = np.array(y_kt + s_up_kt + s_down_kt)
        c = np.array(cost_coeffs)
        penalty_value = penalty*(ys_p.x + ys_n.x)
        v_opt_kt = c.dot(obj_var_opt) + penalty_value

        v_opt_k[-1] += v_opt_kt

        x_trial = np.array([x[g].x for g in range(n_gens)])

        solution = {
            "x": x_trial,
            "y": y_kt,
            "z": z_kt
        }

        solution_storage.add_solution(i, k, t, solution)

        model.printAttr("X")


########################################################################################################################
# Statistical upper bound
########################################################################################################################
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 8 columns and 14 nonzeros
Model fingerprint: 0x9dd89551
Variable types: 5 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 1e+06]
Presolve removed 7 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 7.000000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     700.0000000  700.00000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 ava

In [83]:
solutions_df = solution_storage.to_dataframe()
solutions_df

x        y      z
i k t                       
0 0 0  [1.0]  [350.0]  [0.0]
    1  [1.0]  [200.0]  [1.0]
  1 0  [1.0]  [350.0]  [0.0]
    1  [1.0]  [300.0]  [1.0]

### Backward pass

In [84]:
########################################################################################################################
# Backward pass
########################################################################################################################

binarizer = utils.Binarizer()


x = np.empty(n_gens, dtype=object)
y = np.empty(n_gens, dtype=object)
z = np.empty(n_gens, dtype=object)
s_up = np.empty(n_gens, dtype=object)
s_down = np.empty(n_gens, dtype=object)
theta = None
ys_p = None
ys_n = None


cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist()

penalty = 10000

# Cut variable upper bounds
theta_lb = -100000


samples = [[0,1], [0,0]]
n_samples = len(samples)
v_opt_k =[]

# Solution storage
dual_multipliers_storage = storage.SolutionStorage()
cut_coefficients_storage = storage.SolutionStorage()
i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    for t in reversed(range(n_stages)):
        n_realizations = n_nodes_per_stage[t]
        dual_solutions = {
            "opt_values": [],
            "opt_multipliers": []
            }
        cut_coefficients = {
            "gradients": [], 
            "intercepts": []
        }
        
        for n in range(n_realizations):
            
            # Binarization
            x_lambdas = np.array([])
            x_bin_multipliers = np.array([[]])
            if t>0: 
                x_vars_float = solution_storage.get_solution(i,k,t-1)["x"]
            else:
                x_vars_float = np.zeros(len(x_vars_float))
            for j in range(len(x_vars_float)):
                bin_vars, bin_multipliers = binarizer.binary_expansion(x_vars_float[j], upper_bound=1, precision=0.5)
                x_lambdas = np.concatenate((x_lambdas, bin_vars), axis = None)
                multipliers = np.zeros((len(x_vars_float), len(bin_multipliers)))
                multipliers[j] = bin_multipliers
                x_bin_multipliers = np.concatenate((x_bin_multipliers, multipliers), axis = 1)
                
            n_x_bin_multipliers = x_bin_multipliers.shape[1]
            kappa_x = np.empty(n_x_bin_multipliers, dtype=object)
            #dual_multipliers = np.empty(n_x_bin_multipliers, dtype=object)
            
            model = gp.Model("D{}".format(t))

            #Variables
            for g in range(n_gens):
                x[g] = model.addVar(vtype = gp.GRB.BINARY, name = "x_%i,%i"%(g+1,t+1))
                y[g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "y_%i,%i"%(g+1,t+1))
                z[g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, ub = 1, name = "y_%i,%i"%(g+1,t+1))
                s_up[g] = model.addVar(vtype = gp.GRB.BINARY, name = "s_up_%i,%i"%(g+1,t+1))
                s_down[g] = model.addVar(vtype = gp.GRB.BINARY, name = "s_down_%i,%i"%(g+1,t+1))
            
            for m in range(n_x_bin_multipliers):
                kappa_x[m] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, ub = 1, name = "kappa_%i"%(m+1))
            
            theta = model.addVar(vtype = gp.GRB.CONTINUOUS, name = "theta_%i"%(t+1))
            ys_p = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "ys_p_%i"%(t+1))
            ys_n = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "ys_n_%i"%(t+1))

            model.update()

            # Objective
            obj_vars = y.tolist() + s_up.tolist() + s_down.tolist()
            objective_terms = gp.LinExpr(cost_coeffs, obj_vars)
            objective_terms += penalty*(ys_p+ ys_n)
            objective_terms += theta

            relaxed_terms = []
            relaxed_terms = [x_lambdas[m] - kappa_x[m] for m in range(n_x_bin_multipliers)]
            #total_objective = objective + quicksum(relaxed_terms[m]*dual_multipliers[m] for m in range(n_x_bin_multipliers))
            #model.setObjective(total_objective)

            # Balance constraints
            model.addConstr((gp.quicksum(y) + ys_p - ys_n == p_d[t][n]), "balance")

            # Generator constraints
            model.addConstrs((y[g] >= pg_min[g]*x[g] for g in range(n_gens)), "gen_min")
            model.addConstrs((y[g] <= pg_max[g]*x[g] for g in range(n_gens)), "gen_max")

            # Power flow constraints
            model.addConstrs((gp.quicksum(ptdf[l,b] * gp.quicksum(y[g] for g in gens_at_bus[b]) for b in range(n_buses)) 
                <= pl_max for l in range(n_lines)), "flow(1)")
            model.addConstrs((-gp.quicksum(ptdf[l,b] * gp.quicksum(y[g] for g in gens_at_bus[b]) for b in range(n_buses)) 
                <= pl_max for l in range(n_lines)), "flow(2)")

            # Startup and shutdown constraints    
            model.addConstrs((x[g] - z[g] <= s_up[g]  for g in range(n_gens)), "up-down(1)")
            model.addConstrs((x[g] - z[g] <= s_up[g] - s_down[g]  for g in range(n_gens)), "up-down(2)")

            # TODO Ramp rate constraints

            # Copy constraints
            model.addConstrs((z[g] == gp.quicksum(x_bin_multipliers[g,j]*kappa_x[j] for j in range(n_x_bin_multipliers)) 
                for g in range(n_gens)), "copy")

            # Cut constraints
            if not t == n_stages-1:
                cut_coeffs = cut_coefficients_storage.get_stage_solutions(t+1)
                gradients = cut_coefficients["gradients"]
                intercepts = cut_coefficients["intercepts"]
                model.addConstr((theta >= theta_lb), "cut")
                model.addConstrs((theta >= intcp + grad.dot(x) for intcp, grad in zip(intercepts, gradients)), "cut")

            model.update()


            sg_method = dualsolver.SubgradientMethod(max_iterations=100)

            model, sg_results = sg_method.solve(model, objective_terms, relaxed_terms, 10000)
            model.printAttr("X")

            #model.optimize()
            #model.display()         
            
            # Dual multipliers
            dual_solutions["opt_values"].append(sg_results.obj_value)
            dual_solutions["opt_multipliers"].append(sg_results.multipliers)
            
        
        dual_multipliers_storage.add_solution(i, k, t, dual_solutions)

        # Calculate and store cut coefficients
        probabilities = probs[t]
        intercept = np.array(probabilities).dot(np.array(dual_solutions["opt_values"]))
        gradient = np.array(probabilities).dot(np.array(dual_solutions["opt_multipliers"]))
        
        cut_coefficients["intercepts"].append(intercept)
        cut_coefficients["gradients"].append(gradient)

        cut_coefficients_storage.add_solution(i,k,t, cut_coefficients)
                

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 10 columns and 16 nonzeros
Model fingerprint: 0x02a4cd6c
Variable types: 7 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [5e-01, 6e+02]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 1e+06]
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 6.000000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     600.0000000  600.00000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 av